<a href="https://colab.research.google.com/github/JyothyVariyampat/Benchmark-ML-and-DL-Models-for-Small-Molecule-Immunity-Target-Bioactivity-Prediction./blob/main/FFNN_AID1578.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
"""Deepclassifier-aid1239_revised.ipynb"""

import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, f1_score, cohen_kappa_score, precision_score, recall_score
from google.colab import drive


In [ ]:
# Ensure reproducibility
import random
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# Load the dataset
#df = pd.read_csv('/content/drive/MyDrive/Datasets/AID_1239/AID1239_extracted_descriptor_data.csv')
#/content/drive/MyDrive/Datasets/AID_1239/concatenated_AID_1239_Active_Inactive.csv
df = pd.read_csv('/content/drive/MyDrive/Datasets/AID_1578/AID1578_RDDESCR4000ACTIVE_INACTIVE_activity.CSV')
df_copy = df.copy()


<ipython-input-6-ccf8f2b4e056>:4: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,2

In [ ]:
df.shape

(4000, 16384)

In [ ]:
# prompt: value counts for PUBCHEM_ACTIVITY_OUTCOME

df['PUBCHEM_ACTIVITY_OUTCOME'].value_counts()

,count
PUBCHEM_ACTIVITY_OUTCOME,
Active,2000
Inactive,2000


In [ ]:
# EDA 1
# Calculate the percentage of NaN values in each column


nan_percentage = df.isna().mean() * 100

# Find columns where more than 50% of the values are NaN
columns_with_nan_above_50 = nan_percentage[nan_percentage > 50].index.tolist()

print(f"Number of columns with more than 50% NaN values: {len(columns_with_nan_above_50)}")
print("Columns:", columns_with_nan_above_50)


Number of columns with more than 50% NaN values: 691
Columns: ['morgan_counts_1415881739', 'morgan_counts_2327781401', 'morgan_counts_518558748', 'morgan_counts_2998668329', 'morgan_counts_613485615', 'morgan_counts_4219102253', 'morgan_counts_2788108361', 'morgan_counts_3892164686', 'morgan_counts_613493838', 'morgan_counts_670698601', 'morgan_counts_1575699570', 'morgan_counts_2358897784', 'morgan_counts_3766531224', 'morgan_counts_594373804', 'morgan_counts_2184004791', 'morgan_counts_2784088280', 'morgan_counts_3198227675', 'morgan_counts_2457856221', 'morgan_counts_3928079582', 'morgan_counts_3492268261', 'morgan_counts_2133102822', 'morgan_counts_3977905387', 'morgan_counts_3856818424', 'morgan_counts_614175997', 'morgan_counts_1767681279', 'morgan_counts_4019123456', 'morgan_counts_736352522', 'morgan_counts_1237148938', 'morgan_counts_3044949264', 'morgan_counts_2749245717', 'morgan_counts_476332314', 'morgan_counts_4048680257', 'morgan_counts_3821068628', 'morgan_counts_222930

In [ ]:
# EDA 2
# Calculate the percentage of zero values in each column



zero_percentage = (df == 0).mean() * 100

# Find columns where more than 50% of the values are 0
columns_with_zero_above_50 = zero_percentage[zero_percentage > 50].index.tolist()

print(f"Number of columns with more than 50% zero values: {len(columns_with_zero_above_50)}")
print("Columns:", columns_with_zero_above_50)


Number of columns with more than 50% zero values: 2889
Columns: ['rdf28', 'rdf29', 'rdf57', 'rdf58', 'rdf59', 'rdf87', 'rdf88', 'rdf89', 'rdf118', 'rdf119', 'rdf147', 'rdf148', 'rdf149', 'rdf178', 'rdf179', 'rdf208', 'rdf209', 'maccs0', 'maccs1', 'maccs2', 'maccs3', 'maccs4', 'maccs5', 'maccs6', 'maccs7', 'maccs8', 'maccs9', 'maccs10', 'maccs11', 'maccs12', 'maccs13', 'maccs14', 'maccs15', 'maccs16', 'maccs17', 'maccs18', 'maccs19', 'maccs20', 'maccs21', 'maccs22', 'maccs23', 'maccs24', 'maccs25', 'maccs26', 'maccs27', 'maccs28', 'maccs29', 'maccs30', 'maccs31', 'maccs32', 'maccs33', 'maccs34', 'maccs35', 'maccs36', 'maccs37', 'maccs38', 'maccs39', 'maccs40', 'maccs41', 'maccs42', 'maccs43', 'maccs44', 'maccs45', 'maccs46', 'maccs47', 'maccs48', 'maccs49', 'maccs50', 'maccs51', 'maccs52', 'maccs53', 'maccs54', 'maccs55', 'maccs56', 'maccs57', 'maccs58', 'maccs59', 'maccs60', 'maccs61', 'maccs62', 'maccs63', 'maccs64', 'maccs66', 'maccs67', 'maccs68', 'maccs69', 'maccs70', 'maccs71', 'm

In [ ]:
# prompt: # # Replace hyphens with NaN

# Replace hyphens with NaN
df = df.replace('-', np.nan)

In [ ]:
# Remove all the columns which falls in this categories:
# Number of columns with more than 50% NaN values:
# Number of columns with more than 50% zero values:

import pandas as pd

# Assuming 'data' is your DataFrame

# 1. Identify columns with more than 50% NaN values
threshold_nan = len(df) * 0.5
columns_with_nan = df.columns[df.isna().sum() > threshold_nan]

# 2. Identify columns with more than 50% zero values
threshold_zero = len(df) * 0.5
columns_with_zero = df.columns[(df == 0).sum() > threshold_zero]

# 3. Combine both sets of columns to drop
columns_to_drop = set(columns_with_nan).union(set(columns_with_zero))

# 4. Drop these columns from the DataFrame
data_cleaned = df.drop(columns=columns_to_drop)

# Check the shape of the new DataFrame to confirm the columns have been removed
print(f"Original shape: {df.shape}")
print(f"New shape after dropping columns: {data_cleaned.shape}")


Original shape: (4000, 16384)
New shape after dropping columns: (4000, 12773)


In [ ]:
df = data_cleaned.copy()

In [ ]:
# Step 1: Fill null values with zero
df.fillna(0, inplace=True)
df = df.round(3)


In [ ]:
# Save the preprocessed dataset for future reference

# /content/drive/MyDrive/Datasets/AID_1259354/AID1259354_RDDESCR3650ACTIVE_INACTIVE.CSV
# /content/drive/MyDrive/Datasets/AID_1578/AID1578_RDDESCR4000ACTIVE_INACTIVE_activity.CSV
df.to_csv('/content/drive/MyDrive/Datasets/AID_1578/preprocessed_AID_1578_12773Columns_Active_Inactive.csv', index=False)
print(df.head())

                                              SMILES MOLECULEID autocorr2d0  \
0   CC1=C(C2=C(C=C(O)C=C2)N1C1=CC=CC=C1)[N+]([O-])=O   M2624390       3.264   
1  CC(=O)C1=CC=C(OC2=C(C=C(C=C2)C(=O)C2=CC(C)=C(C...   M3480558       3.589   
2  ClCC(=O)N(C(C(=O)NC1CCCCC1)C1=CN=CC=C1)C1=CC(C...   M1253430       3.655   
3       Cl.CN(C)C1=CC=C(NC2=NC3=CC=CC=C3C(N)=N2)C=C1   M3516870       3.301   
4                ClCC(=O)C1=CC2=C3N(CCC2)C(=O)CC3=C1   M1206249       3.183   

  autocorr2d1 autocorr2d2 autocorr2d3 autocorr2d4 autocorr2d5 autocorr2d6  \
0       3.641       3.719       3.762       3.625        3.42       3.038   
1        3.95        4.06       3.984       3.896       3.903       3.752   
2       4.003       4.144         4.2       4.388       4.487        4.42   
3        3.64       3.673       3.528       3.289        3.13       3.071   
4       3.572       3.696       3.536       3.292       3.086       2.887   

  autocorr2d7  ... torsions_((C,3,1),(C,3,1),(C,3,1),(O,1,1)) 

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Datasets/AID_1578/preprocessed_AID_1578_12773Columns_Active_Inactive.csv')

In [ ]:
df.head()

,SMILES,MOLECULEID,autocorr2d0,autocorr2d1,autocorr2d2,autocorr2d3,autocorr2d4,autocorr2d5,autocorr2d6,autocorr2d7,...,"torsions_((C,3,1),(C,3,1),(C,3,1),(O,1,1))","torsions_((C,4,0),(C,3,1),(C,3,1),(C,3,1))","torsions_((O,1,1),(C,3,1),(C,4,0),(Cl,1,0))","torsions_((C,3,1),(C,3,1),(C,3,1),(N,3,0))","torsions_((C,3,1),(C,3,1),(C,3,1),(N,2,1))",morgan_counts_2834401467,morgan_counts_1492260254,morgan_counts_4255104684,morgan_counts_12835993,PUBCHEM_ACTIVITY_OUTCOME
0,CC1=C(C2=C(C=C(O)C=C2)N1C1=CC=CC=C1)[N+]([O-])=O,M2624390,3.264,3.641,3.719,3.762,3.625,3.420,3.038,2.059,...,0,1,0.0,0,0,0.0,0.0,0.0,0.0,Active
1,CC(=O)C1=CC=C(OC2=C(C=C(C=C2)C(=O)C2=CC(C)=C(C...,M3480558,3.589,3.950,4.060,3.984,3.896,3.903,3.752,3.665,...,6,6,0.0,0,0,0.0,0.0,0.0,0.0,Active
2,ClCC(=O)N(C(C(=O)NC1CCCCC1)C1=CN=CC=C1)C1=CC(C...,M1253430,3.655,4.003,4.144,4.200,4.388,4.487,4.420,4.029,...,0,1,1.0,2,2,0.0,0.0,0.0,0.0,Active
3,Cl.CN(C)C1=CC=C(NC2=NC3=CC=CC=C3C(N)=N2)C=C1,M3516870,3.301,3.640,3.673,3.528,3.289,3.130,3.071,2.973,...,0,0,0.0,6,5,0.0,0.0,0.0,0.0,Active
4,ClCC(=O)C1=CC2=C3N(CCC2)C(=O)CC3=C1,M1206249,3.183,3.572,3.696,3.536,3.292,3.086,2.887,2.218,...,2,6,1.0,2,0,0.0,0.0,0.0,0.0,Active


In [ ]:
# # Step 1: Replace infinite values with a large finite value
# df.replace([np.inf, -np.inf], np.finfo(np.float64).max, inplace=True)

# # Step 2: Replace hyphens with NaN
# df.replace('-', np.nan, inplace=True)


In [ ]:
# Define target and columns to drop
target_column = 'PUBCHEM_ACTIVITY_OUTCOME'
drop_columns = [ 'SMILES', 'MOLECULEID']


In [ ]:
# Drop unnecessary columns
df.drop(columns=drop_columns, inplace=True)


In [ ]:
# Encode target column (Active/Inactive)
label_encoder = LabelEncoder()
df[target_column] = label_encoder.fit_transform(df[target_column])

In [ ]:
# Split the features and target
X = df.drop(columns=[target_column]).values
y = df[target_column].values

In [ ]:
# prompt: convert all the X to float


# # Convert all columns in X to numeric, coercing errors to NaN
# X = pd.DataFrame(X).apply(pd.to_numeric, errors='coerce').values

# # Fill NaN values resulting from the conversion (if any)
# X = np.nan_to_num(X)

In [ ]:
# Step 2: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# prompt: # Ensure X_train, X_test, y_train, y_test are of numeric type before checking for infinity
# # Ensure X_train and X_test are of numeric type before checking for infinity
# X_train = X_train.astype(np.float64)
# X_test = X_test.astype(np.float64)
# and for  y_train, y_test
# # Check for NaN and infinity values, replace if needed
# X_train[np.isinf(X_train)] = np.nan
# X_train = np.nan_to_num(X_train, nan=np.nanmax(X_train))
# X_test[np.isinf(X_test)] = np.nan
# X_test = np.nan_to_num(X_test, nan=np.nanmax(X_test))
# for y_train, y_test
# and finally replace all the NaN with 0 and
#  null with 0

# Ensure X_train, X_test, y_train, y_test are of numeric type before checking for infinity
X_train = X_train.astype(np.float64)
X_test = X_test.astype(np.float64)
y_train = y_train.astype(np.float64)
y_test = y_test.astype(np.float64)

# Check for NaN and infinity values, replace if needed
X_train[np.isinf(X_train)] = np.nan
X_train = np.nan_to_num(X_train, nan=np.nanmax(X_train))
X_test[np.isinf(X_test)] = np.nan
X_test = np.nan_to_num(X_test, nan=np.nanmax(X_test))

y_train[np.isinf(y_train)] = np.nan
y_train = np.nan_to_num(y_train, nan=np.nanmax(y_train))
y_test[np.isinf(y_test)] = np.nan
y_test = np.nan_to_num(y_test, nan=np.nanmax(y_test))

# Replace NaN with 0
X_train = np.nan_to_num(X_train, nan=0)
X_test = np.nan_to_num(X_test, nan=0)
y_train = np.nan_to_num(y_train, nan=0)
y_test = np.nan_to_num(y_test, nan=0)

In [ ]:
# # Check for NaN and infinity values, replace if needed
# X_train[np.isinf(X_train)] = np.nan
# X_train = np.nan_to_num(X_train, nan=np.nanmax(X_train))

# X_test[np.isinf(X_test)] = np.nan
# X_test = np.nan_to_num(X_test, nan=np.nanmax(X_test))

In [ ]:
# Step 3: Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
# Define the model
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(X_train.shape[1],), kernel_regularizer=l2(0.001)))
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.001)))
model.add(Dropout(0.3))
model.add(Dense(32, activation='relu', kernel_regularizer=l2(0.001)))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# Compile the model
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the model with early stopping
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=64, callbacks=[early_stopping])


Epoch 1/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 5s 37ms/step - accuracy: 0.5566 - loss: 1.5885 - val_accuracy: 0.6862 - val_loss: 1.0909
Epoch 2/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6363 - loss: 1.4685 - val_accuracy: 0.6475 - val_loss: 1.1456
Epoch 3/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6584 - loss: 1.3297 - val_accuracy: 0.6775 - val_loss: 1.0925
Epoch 4/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7091 - loss: 1.1717 - val_accuracy: 0.6975 - val_loss: 1.0524
Epoch 5/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7454 - loss: 1.0702 - val_accuracy: 0.7125 - val_loss: 1.0572
Epoch 6/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8058 - loss: 1.0213 - val_accuracy: 0.7212 - val_loss: 1.0452
Epoch 7/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8309 - loss: 0.9336 - val_accuracy: 0.7337 - val_loss: 1.0625
Epoch 8/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8539 - loss: 0.8289 - val_accuracy: 0.7212 - val_loss

In [ ]:
# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)


In [ ]:
# Evaluate the model on training data
train_loss, train_acc = model.evaluate(X_train, y_train, verbose=0)

In [ ]:
# Make predictions on training data
y_train_pred = model.predict(X_train).flatten()
y_train_pred_classes = (y_train_pred > 0.5).astype(int)

100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


In [ ]:
# Make predictions
y_pred = model.predict(X_test).flatten()
y_pred_classes = (y_pred > 0.5).astype(int)


25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 


In [ ]:
# Calculate metrics for training data
train_roc_auc = roc_auc_score(y_train, y_train_pred)
train_f1 = f1_score(y_train, y_train_pred_classes)
train_kappa = cohen_kappa_score(y_train, y_train_pred_classes)
train_precision = precision_score(y_train, y_train_pred_classes)
train_recall = recall_score(y_train, y_train_pred_classes)

In [ ]:
# Calculate metrics
test_roc_auc = roc_auc_score(y_test, y_pred)
test_f1 = f1_score(y_test, y_pred_classes)
test_kappa = cohen_kappa_score(y_test, y_pred_classes)
test_precision = precision_score(y_test, y_pred_classes)
test_recall = recall_score(y_test, y_pred_classes)


In [ ]:
# Output the results
results = {
    "train_accuracy": train_acc,
    "train_roc_auc": train_roc_auc,
    "train_f1": train_f1,
    "train_kappa": train_kappa,
    "train_precision": train_precision,
    "train_recall": train_recall,
    "test_accuracy": test_acc,
    "test_roc_auc": test_roc_auc,
    "test_f1": test_f1,
    "test_kappa": test_kappa,
    "test_precision": test_precision,
    "test_recall": test_recall
}

print("Model Evaluation Metrics:")
for key, value in results.items():
    print(f"{key}: {value:.4f}")

Model Evaluation Metrics:
train_accuracy: 0.8919
train_roc_auc: 0.9651
train_f1: 0.8943
train_kappa: 0.7837
train_precision: 0.8867
train_recall: 0.9020
test_accuracy: 0.7212
test_roc_auc: 0.7964
test_f1: 0.7137
test_kappa: 0.4426
test_precision: 0.6933
test_recall: 0.7354


**Below is Code with Optuna**

In [ ]:
!pip install --quiet optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.8 MB/s eta 0:00:00


In [ ]:
import optuna
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, f1_score, cohen_kappa_score, precision_score, recall_score
from tensorflow.keras.callbacks import EarlyStopping

# Define the Optuna objective function
def objective(trial):
    # Hyperparameter suggestions
    n_layers = trial.suggest_int("n_layers", 2, 4)
    units = trial.suggest_int("units", 32, 256, step=32)
    dropout_rate = trial.suggest_float("dropout_rate", 0.1, 0.5, step=0.1)
    l2_reg = trial.suggest_float("l2_reg", 1e-5, 1e-2, log=True)
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-2, log=True)
    batch_size = trial.suggest_categorical("batch_size", [32, 64, 128])

    # Build the model
    model = Sequential()
    model.add(Dense(units, activation='relu', input_shape=(X_train.shape[1],), kernel_regularizer=l2(l2_reg)))
    model.add(Dropout(dropout_rate))
    for _ in range(n_layers - 1):
        model.add(Dense(units // 2, activation='relu', kernel_regularizer=l2(l2_reg)))
        model.add(Dropout(dropout_rate))
        units //= 2
    model.add(Dense(1, activation='sigmoid'))

    # Compile the model
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

    # Early stopping
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    # Train the model
    history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50,
                        batch_size=batch_size, callbacks=[early_stopping], verbose=0)

    # Evaluate the model on test data
    test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)

    # Calculate additional metrics on test data
    y_pred = model.predict(X_test).flatten()
    y_pred_classes = (y_pred > 0.5).astype(int)

    if len(np.unique(y_test)) < 2:
        test_roc_auc = None
    else:
        test_roc_auc = roc_auc_score(y_test, y_pred)

    test_f1 = f1_score(y_test, y_pred_classes)
    test_kappa = cohen_kappa_score(y_test, y_pred_classes)
    test_precision = precision_score(y_test, y_pred_classes)
    test_recall = recall_score(y_test, y_pred_classes)

    # Return the test accuracy as the objective to maximize
    return test_acc

# Run Optuna for hyperparameter optimization
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=30)

# Best hyperparameters from Optuna
best_params = study.best_params
print("Best hyperparameters:", best_params)

# Build the final model with the best hyperparameters
n_layers = best_params["n_layers"]
units = best_params["units"]
dropout_rate = best_params["dropout_rate"]
l2_reg = best_params["l2_reg"]
learning_rate = best_params["learning_rate"]
batch_size = best_params["batch_size"]

model = Sequential()
model.add(Dense(units, activation='relu', input_shape=(X_train.shape[1],), kernel_regularizer=l2(l2_reg)))
model.add(Dropout(dropout_rate))
for _ in range(n_layers - 1):
    model.add(Dense(units // 2, activation='relu', kernel_regularizer=l2(l2_reg)))
    model.add(Dropout(dropout_rate))
    units //= 2
model.add(Dense(1, activation='sigmoid'))

# Compile the model
optimizer = Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the final model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50,
                    batch_size=batch_size, callbacks=[early_stopping], verbose=1)

# Evaluate the final model
train_loss, train_acc = model.evaluate(X_train, y_train, verbose=0)
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)

# Predictions
y_train_pred = model.predict(X_train).flatten()
y_train_pred_classes = (y_train_pred > 0.5).astype(int)
y_pred = model.predict(X_test).flatten()
y_pred_classes = (y_pred > 0.5).astype(int)

# Train metrics
train_roc_auc = roc_auc_score(y_train, y_train_pred)
train_f1 = f1_score(y_train, y_train_pred_classes)
train_kappa = cohen_kappa_score(y_train, y_train_pred_classes)
train_precision = precision_score(y_train, y_train_pred_classes)
train_recall = recall_score(y_train, y_train_pred_classes)

# Test metrics
test_roc_auc = roc_auc_score(y_test, y_pred)
test_f1 = f1_score(y_test, y_pred_classes)
test_kappa = cohen_kappa_score(y_test, y_pred_classes)
test_precision = precision_score(y_test, y_pred_classes)
test_recall = recall_score(y_test, y_pred_classes)

# Output the results
results = {
    "train_accuracy": train_acc,
    "train_roc_auc": train_roc_auc,
    "train_f1": train_f1,
    "train_kappa": train_kappa,
    "train_precision": train_precision,
    "train_recall": train_recall,
    "test_accuracy": test_acc,
    "test_roc_auc": test_roc_auc,
    "test_f1": test_f1,
    "test_kappa": test_kappa,
    "test_precision": test_precision,
    "test_recall": test_recall
}

print("\nModel Evaluation Metrics:")
for key, value in results.items():
    print(f"{key}: {value:.4f}")


[I 2024-11-19 17:42:48,909] A new study created in memory with name: no-name-fadaee20-40da-4732-94a3-7de40049a7d0
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step  


[I 2024-11-19 17:42:57,225] Trial 0 finished with value: 0.7250000238418579 and parameters: {'n_layers': 2, 'units': 160, 'dropout_rate': 0.1, 'l2_reg': 3.611254271885216e-05, 'learning_rate': 0.0014921371926482142, 'batch_size': 32}. Best is trial 0 with value: 0.7250000238418579.
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step  


[I 2024-11-19 17:43:08,711] Trial 1 finished with value: 0.731249988079071 and parameters: {'n_layers': 2, 'units': 128, 'dropout_rate': 0.30000000000000004, 'l2_reg': 0.009078045143400796, 'learning_rate': 0.00014833966954244643, 'batch_size': 64}. Best is trial 1 with value: 0.731249988079071.
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


...

25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step  


[I 2024-11-19 17:48:01,607] Trial 29 finished with value: 0.7162500023841858 and parameters: {'n_layers': 2, 'units': 160, 'dropout_rate': 0.1, 'l2_reg': 1.012273515737187e-05, 'learning_rate': 0.00029091758909005437, 'batch_size': 32}. Best is trial 19 with value: 0.7412499785423279.


Best hyperparameters: {'n_layers': 3, 'units': 64, 'dropout_rate': 0.2, 'l2_reg': 6.05747321230457e-05, 'learning_rate': 0.00015487245470113844, 'batch_size': 128}


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 56ms/step - accuracy: 0.5289 - loss: 0.7938 - val_accuracy: 0.6425 - val_loss: 0.6472
Epoch 2/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6699 - loss: 0.6481 - val_accuracy: 0.6488 - val_loss: 0.6453
Epoch 3/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7265 - loss: 0.5553 - val_accuracy: 0.6750 - val_loss: 0.6068
Epoch 4/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7573 - loss: 0.5074 - val_accuracy: 0.6662 - val_loss: 0.6060
Epoch 5/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7840 - loss: 0.4704 - val_accuracy: 0.7025 - val_loss: 0.6036
Epoch 6/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8113 - loss: 0.4529 - val_accuracy: 0.6950 - val_loss: 0.6076
Epoch 7/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8368 - loss: 0.3947 - val_accuracy: 0.7075 - val_loss: 0.5990
Epoch 8/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8510 - loss: 0.3527 - val_accuracy: 0.7100 - val_loss